In [2]:
# Constant variables definitions
DIR_PATH = "sample_texts/"




In [3]:
import os

In [9]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

import nltk


In [37]:
def process_file(fname):
    file_path = DIR_PATH + fname
    print(file_path)
    
    content = ""

    with open(file_path, 'r') as f:

        content = f.readlines()
        
    content = "".join(content)
    return content


In [38]:
# Below are the following files that will be tested.

files = os.listdir(DIR_PATH)
files


['sample1.txt', 'sample2.txt', 'sample3.txt']

In [42]:
# nltk.download('stopwords')
# nltk.download('punky')

[nltk_data] Downloading package stopwords to /Users/jaime/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [44]:
eng_stops = nltk.corpus.stopwords.words('english')
symbolic_stops = ['.', ',', '!', '?', ';', ':', '[', ']']

eng_stops += symbolic_stops

In [45]:
# symbolic_stopwords = ['.', ',', ]

# sample_index = 0
# file_chosen = files[sample_index]

# words = process_file(file_chosen)

# nltk.word_tokenize(words)


# nltk.stem.snowball.SnowballStemmer("english")